<a href="https://colab.research.google.com/github/cherskiberris/Encuentros_Sincronos/blob/main/Generador_de_Cronogramas_Escolares_(con_exportaci%C3%B3n_a_Excel).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import calendar
import datetime
import pandas as pd
import os
from collections import defaultdict
import random # Importar la biblioteca random

# Nota: Este script requiere la biblioteca pandas.
# Puedes instalarla con el comando: pip install pandas openpyxl

def generar_cronograma_mensual(materias, anio, mes, contador_encuentros):
    """
    Genera un cronograma de encuentros para un mes específico, actualizando el contador.

    Args:
        materias (list): Lista de materias a agendar para el mes.
        anio (int): El año para el cual se generará el cronograma.
        mes (int): El mes (en número) para el cual se generará el cronograma.
        contador_encuentros (defaultdict): Diccionario para llevar la cuenta de los
                                           encuentros por materia.

    Returns:
        tuple: Una tupla conteniendo dos listas:
               - cronograma_mes (list): Los encuentros agendados para el mes.
               - sin_agendar_mes (list): Las materias que no pudieron ser agendar.
    """
    # --- 1. Definición de Horarios y Constantes ---
    HORAS_DISPONIBLES = [9, 10, 11, 13, 14]
    DIAS_SEMANA = ["Lunes", "Martes", "Miércoles", "Jueves", "Viernes"]

    # --- 2. Inicialización de Estructuras de Datos para el mes ---
    cronograma_mes = []
    sin_agendar_mes = []
    reuniones_instructor_dia = {}
    reuniones_grado_dia = {}
    horarios_ocupados = {}

    # --- 3. Generar la Lista de Días Hábiles del Mes ---
    dias_habiles_del_mes = []
    cal = calendar.Calendar()
    for dia in cal.itermonthdates(anio, mes):
        if dia.month == mes and dia.weekday() < 5:
            dias_habiles_del_mes.append(dia)

    # --- Barajar la lista de días hábiles para distribuir los encuentros ---
    random.shuffle(dias_habiles_del_mes)

    # --- 4. Algoritmo de Asignación Mensual ---
    for materia_info in materias:
        instructor = materia_info['instructor']
        grado = materia_info['grado']
        materia_nombre = materia_info['materia']

        encuentro_agendado = False

        # Iterar sobre los días hábiles (ahora barajados)
        for dia in dias_habiles_del_mes:
            dia_str = dia.strftime('%Y-%m-%d')
            for hora in HORAS_DISPONIBLES:

                if reuniones_instructor_dia.get(instructor, {}).get(dia_str, 0) >= 2:
                    continue
                if reuniones_grado_dia.get(grado, {}).get(dia_str, 0) >= 1:
                    continue
                if hora in horarios_ocupados.get(dia_str, []):
                    continue

                # --- Asignación del Encuentro ---
                # Se encontró un espacio. Se procede a agendar.

                # Incrementar y obtener el número de encuentro para esta materia
                llave_materia = (grado, materia_nombre)
                contador_encuentros[llave_materia] += 1
                numero_encuentro = contador_encuentros[llave_materia]

                cronograma_mes.append({
                    'fecha': dia_str,
                    'dia_semana': DIAS_SEMANA[dia.weekday()],
                    'hora_inicio': f"{hora:02d}:00",
                    'hora_fin': f"{hora+1:02d}:00",
                    'instructor': instructor,
                    'materia': materia_nombre,
                    'grado': grado,
                    'encuentro_nro': numero_encuentro
                })

                # Actualizar estructuras de control
                reuniones_instructor_dia.setdefault(instructor, {}).setdefault(dia_str, 0)
                reuniones_instructor_dia[instructor][dia_str] += 1
                reuniones_grado_dia.setdefault(grado, {}).setdefault(dia_str, 0)
                reuniones_grado_dia[grado][dia_str] += 1
                horarios_ocupados.setdefault(dia_str, []).append(hora)

                encuentro_agendado = True
                break

            if encuentro_agendado:
                break

        if not encuentro_agendado:
            sin_agendar_mes.append({**materia_info, 'mes_fallido': f"{mes}-{anio}"})

    return cronograma_mes, sin_agendar_mes

def imprimir_cronograma_anual(cronograma, sin_agendar, anio_inicio, mes_inicio, anio_fin, mes_fin):
    """Imprime el cronograma anual de forma legible en la consola."""
    periodo = f"{mes_inicio}/{anio_inicio} - {mes_fin}/{anio_fin}"
    print("\n" + "="*80)
    print(f"CRONOGRAMA ANUAL DE ENCUENTROS SINCRÓNICOS - PERÍODO {periodo}")
    print("="*80)

    if not cronograma:
        print("No se pudo agendar ningún encuentro.")
        return

    cronograma_ordenado = sorted(cronograma, key=lambda x: (x['fecha'], x['hora_inicio']))

    ultimo_mes_impreso = None
    for encuentro in cronograma_ordenado:
        mes_actual = datetime.datetime.strptime(encuentro['fecha'], '%Y-%m-%d').strftime('%B %Y')
        if mes_actual != ultimo_mes_impreso:
            print(f"\n\n--- {mes_actual.upper()} ---")
            ultimo_mes_impreso = mes_actual

        print(
            f"  [{encuentro['fecha']} {encuentro['hora_inicio']}] "
            f"Grado: {encuentro['grado']:<10} | "
            f"Materia: {encuentro['materia']} (Encuentro #{encuentro['encuentro_nro']}) | "
            f"Instructor: {encuentro['instructor']}"
        )

    if sin_agendar:
        print("\n" + "="*80)
        print("ATENCIÓN: Los siguientes encuentros no pudieron ser agendados:")
        print("="*80)
        for item in sin_agendar:
            print(f"- Mes: {item['mes_fallido']} | Grado: {item['grado']}, Materia: {item['materia']}, Instructor: {item['instructor']}")

def exportar_a_excel_anual(cronograma, sin_agendar, anio_inicio, mes_inicio, anio_fin, mes_fin):
    """Exporta el cronograma anual a un archivo Excel."""
    if not cronograma:
        print("\nNo hay datos para exportar a Excel.")
        return

    datos_para_df = []
    for item in cronograma:
        datos_para_df.append({
            'Fecha': item['fecha'],
            'Hora': f"{item['hora_inicio']} - {item['hora_fin']}",
            'Grado': item['grado'],
            'Materia': item['materia'],
            'Encuentro Nro': item['encuentro_nro'],
            'Instructor': item['instructor']
        })

    df_cronograma = pd.DataFrame(datos_para_df)
    df_cronograma = df_cronograma.sort_values(by=['Fecha', 'Hora']).reset_index(drop=True)

    periodo_str = f"{anio_inicio}-{anio_fin if anio_inicio == anio_fin else str(anio_fin)[-2:]}"
    nombre_archivo = f"Cronograma_Anual_{periodo_str}.xlsx"

    with pd.ExcelWriter(nombre_archivo, engine='openpyxl') as writer:
        df_cronograma.to_excel(writer, sheet_name='Cronograma Anual', index=False)
        if sin_agendar:
            df_sin_agendar = pd.DataFrame(sin_agendar)
            df_sin_agendar.to_excel(writer, sheet_name='No Agendados', index=False)

    print("\n" + "="*80)
    print(f"✅ ¡Cronograma anual exportado exitosamente a '{nombre_archivo}'!")
    print("="*80)

# --- EJECUCIÓN PRINCIPAL ---
if __name__ == "__main__":
    lista_de_materias = [
        {'instructor': 'Yosibel Garcia', 'materia': 'Cívica', 'grado': '7mo grado'},
        {'instructor': 'Ivonne Blanco', 'materia': 'Educación Física', 'grado': '7mo grado'},
        {'instructor': 'Osmary Mujica', 'materia': 'Inglés', 'grado': '7mo grado'},
        {'instructor': 'Freddy Martinez', 'materia': 'Matemática', 'grado': '7no grado'},
        {'instructor': 'Ivonne Blanco', 'materia': 'Ciencias', 'grado': '7mo grado'},
        {'instructor': 'Osmary Mujica', 'materia': 'Castellano', 'grado': '7mo grado'},
        {'instructor': 'Yosibel Garcia', 'materia': 'Lideres Emergentes', 'grado': '7mo grado'},
        {'instructor': 'Jhonfrank Sanchez', 'materia': 'Castellano', 'grado': '8vo grado'},
        {'instructor': 'Jhonfrank Sanchez', 'materia': 'Inglés', 'grado': '8vo grado'},
        {'instructor': 'Freddy Martinez', 'materia': 'Matemática', 'grado': '8vo grado'},
        {'instructor': 'Ivonne Blanco', 'materia': 'Ciencias', 'grado': '8vo grado'},
        {'instructor': 'Alvaro Gonzalez', 'materia': 'Finanzas Personales', 'grado': '8vo grado'},
        {'instructor': 'Ivonne Blanco', 'materia': 'Educación para la Salud', 'grado': '8vo grado'},
        {'instructor': 'Osmary Mujica', 'materia': 'Castellano', 'grado': '9no grado'},
        {'instructor': 'Osmary Mujica', 'materia': 'Inglés', 'grado': '9no grado'},
        {'instructor': 'Freddy Martinez', 'materia': 'Álgebra I', 'grado': '9no grado'},
        {'instructor': 'Ivonne Blanco', 'materia': 'Biología', 'grado': '9no grado'},
        {'instructor': 'Yosibel Garcia', 'materia': 'Historia de USA', 'grado': '9no grado'},
        {'instructor': 'Ivonne Blanco', 'materia': 'Educación Física', 'grado': '9no grado'},
        {'instructor': 'Osmary Mujica', 'materia': 'Inglés', 'grado': '10mo grado'},
        {'instructor': 'Freddy Martinez', 'materia': 'Geometría', 'grado': '10mo grado'},
        {'instructor': 'Yosibel Garcia', 'materia': 'Ética', 'grado': '10mo grado'},
        {'instructor': 'Yosibel Garcia', 'materia': 'Arte', 'grado': '10mo grado'},
        {'instructor': 'Alvaro Gonzalez', 'materia': 'Orientación Vocacional', 'grado': '10mo grado'},
        {'instructor': 'Yosibel Garcia', 'materia': 'Historia Univeral', 'grado': '10mo grado'},
        {'instructor': 'Yosibel Garcia', 'materia': 'Actuación', 'grado': '10mo grado'},
        {'instructor': 'Jhonfrank Sanchez', 'materia': 'Castellano', 'grado': '11vo grado'},
        {'instructor': 'Osmary Mujica', 'materia': 'Inglés', 'grado': '11vo grado'},
        {'instructor': 'Freddy Martinez', 'materia': 'Algebra II', 'grado': '11vo grado'},
        {'instructor': 'Freddy Martinez', 'materia': 'Física', 'grado': '11vo grado'},
        {'instructor': 'Yosibel Garcia', 'materia': 'Arte', 'grado': '11vo grado'},
        {'instructor': 'Alvaro Gonzalez', 'materia': 'Finanzas Personales y Gestión del Dinero', 'grado': '11vo grado'},
        {'instructor': 'Yosibel Garcia', 'materia': 'Historia del Arte', 'grado': '11vo grado'},
        {'instructor': 'Ivonne Blanco', 'materia': 'Ciencias de la Tierra', 'grado': '11vo grado'},
        {'instructor': 'Jhonfrank Sanchez', 'materia': 'Castellano', 'grado': '12vo grado'},
        {'instructor': 'Jhonfrank Sanchez', 'materia': 'Inglés', 'grado': '12vo grado'},
        {'instructor': 'Freddy Martinez', 'materia': 'Ciencias de la Computación', 'grado': '12vo grado'},
        {'instructor': 'Freddy Martinez', 'materia': 'Física', 'grado': '12vo grado'},
        {'instructor': 'Yosibel Garcia', 'materia': 'Gobierno de USA', 'grado': '12vo grado'},
        {'instructor': 'Alvaro Gonzalez', 'materia': 'Liderazgo', 'grado': '12vo grado'},
        {'instructor': 'Alvaro Gonzalez', 'materia': 'Economía', 'grado': '12vo grado'},
        {'instructor': 'Ivonne Blanco', 'materia': 'Química', 'grado': '12vo grado'}
    ]

    # --- Definir el período del año escolar ---
    ANIO_INICIO, MES_INICIO = 2025, 8  # Agosto 2025
    ANIO_FIN, MES_FIN = 2026, 5      # Mayo 2026

    # --- Preparar estructuras para el ciclo anual ---
    cronograma_anual_completo = []
    no_agendados_total = []
    contador_general_encuentros = defaultdict(int) # Usamos defaultdict para simplificar

    print("Iniciando generación de cronograma para el año escolar...")

    # --- Iterar a través de cada mes del año escolar ---
    anio_actual, mes_actual = ANIO_INICIO, MES_INICIO
    while (anio_actual, mes_actual) <= (ANIO_FIN, MES_FIN):
        print(f"\nProcesando mes: {mes_actual}/{anio_actual}...")

        cronograma_del_mes, no_agendados_del_mes = generar_cronograma_mensual(
            lista_de_materias, anio_actual, mes_actual, contador_general_encuentros
        )

        cronograma_anual_completo.extend(cronograma_del_mes)
        no_agendados_total.extend(no_agendados_del_mes)

        # Avanzar al siguiente mes
        mes_actual += 1
        if mes_actual > 12:
            mes_actual = 1
            anio_actual += 1

    # --- Mostrar y exportar resultados finales ---
    imprimir_cronograma_anual(cronograma_anual_completo, no_agendados_total, ANIO_INICIO, MES_INICIO, ANIO_FIN, MES_FIN)
    exportar_a_excel_anual(cronograma_anual_completo, no_agendados_total, ANIO_INICIO, MES_INICIO, ANIO_FIN, MES_FIN)

Iniciando generación de cronograma para el año escolar...

Procesando mes: 8/2025...

Procesando mes: 9/2025...

Procesando mes: 10/2025...

Procesando mes: 11/2025...

Procesando mes: 12/2025...

Procesando mes: 1/2026...

Procesando mes: 2/2026...

Procesando mes: 3/2026...

Procesando mes: 4/2026...

Procesando mes: 5/2026...

CRONOGRAMA ANUAL DE ENCUENTROS SINCRÓNICOS - PERÍODO 8/2025 - 5/2026


--- AUGUST 2025 ---
  [2025-08-01 09:00] Grado: 12vo grado | Materia: Gobierno de USA (Encuentro #1) | Instructor: Yosibel Garcia
  [2025-08-05 09:00] Grado: 7mo grado  | Materia: Educación Física (Encuentro #1) | Instructor: Ivonne Blanco
  [2025-08-05 10:00] Grado: 8vo grado  | Materia: Inglés (Encuentro #1) | Instructor: Jhonfrank Sanchez
  [2025-08-05 11:00] Grado: 9no grado  | Materia: Inglés (Encuentro #1) | Instructor: Osmary Mujica
  [2025-08-05 13:00] Grado: 10mo grado | Materia: Geometría (Encuentro #1) | Instructor: Freddy Martinez
  [2025-08-05 14:00] Grado: 11vo grado | Materia

In [ ]:
import calendar
import datetime
import pandas as pd
import os
from collections import defaultdict
import random # Importar la biblioteca random

# Nota: Este script requiere la biblioteca pandas.
# Puedes instalarla con el comando: pip install pandas openpyxl

def generar_cronograma_mensual(materias, anio, mes, contador_encuentros):
    """
    Genera un cronograma de encuentros para un mes específico, actualizando el contador.

    Args:
        materias (list): Lista de materias a agendar para el mes.
        anio (int): El año para el cual se generará el cronograma.
        mes (int): El mes (en número) para el cual se generará el cronograma.
        contador_encuentros (defaultdict): Diccionario para llevar la cuenta de los
                                           encuentros por materia.

    Returns:
        tuple: Una tupla conteniendo dos listas:
               - cronograma_mes (list): Los encuentros agendados para el mes.
               - sin_agendar_mes (list): Las materias que no pudieron ser agendar.
    """
    # --- 1. Definición de Horarios y Constantes ---
    HORAS_DISPONIBLES = [9, 10, 11, 13, 14]
    DIAS_SEMANA = ["Lunes", "Martes", "Miércoles", "Jueves", "Viernes"]

    # --- 2. Inicialización de Estructuras de Datos para el mes ---
    cronograma_mes = []
    sin_agendar_mes = []
    reuniones_instructor_dia = {}
    reuniones_grado_dia = {}
    horarios_ocupados = {}

    # --- 3. Generar la Lista de Días Hábiles del Mes ---
    dias_habiles_del_mes = []
    cal = calendar.Calendar()
    for dia in cal.itermonthdates(anio, mes):
        if dia.month == mes and dia.weekday() < 5:
            dias_habiles_del_mes.append(dia)

    # --- Barajar la lista de días hábiles para distribuir los encuentros ---
    random.shuffle(dias_habiles_del_mes)

    # --- 4. Algoritmo de Asignación Mensual ---
    for materia_info in materias:
        instructor = materia_info['instructor']
        grado = materia_info['grado']
        materia_nombre = materia_info['materia']

        encuentro_agendado = False

        # Iterar sobre los días hábiles (ahora barajados)
        for dia in dias_habiles_del_mes:
            dia_str = dia.strftime('%Y-%m-%d')
            for hora in HORAS_DISPONIBLES:

                if reuniones_instructor_dia.get(instructor, {}).get(dia_str, 0) >= 2:
                    continue
                if reuniones_grado_dia.get(grado, {}).get(dia_str, 0) >= 1:
                    continue
                if hora in horarios_ocupados.get(dia_str, []):
                    continue

                # --- Asignación del Encuentro ---
                # Se encontró un espacio. Se procede a agendar.

                # Incrementar y obtener el número de encuentro para esta materia
                llave_materia = (grado, materia_nombre)
                contador_encuentros[llave_materia] += 1
                numero_encuentro = contador_encuentros[llave_materia]

                cronograma_mes.append({
                    'fecha': dia_str,
                    'dia_semana': DIAS_SEMANA[dia.weekday()],
                    'hora_inicio': f"{hora:02d}:00",
                    'hora_fin': f"{hora+1:02d}:00",
                    'instructor': instructor,
                    'materia': materia_nombre,
                    'grado': grado,
                    'encuentro_nro': numero_encuentro
                })

                # Actualizar estructuras de control
                reuniones_instructor_dia.setdefault(instructor, {}).setdefault(dia_str, 0)
                reuniones_instructor_dia[instructor][dia_str] += 1
                reuniones_grado_dia.setdefault(grado, {}).setdefault(dia_str, 0)
                reuniones_grado_dia[grado][dia_str] += 1
                horarios_ocupados.setdefault(dia_str, []).append(hora)

                encuentro_agendado = True
                break

            if encuentro_agendado:
                break

        if not encuentro_agendado:
            sin_agendar_mes.append({**materia_info, 'mes_fallido': f"{mes}-{anio}"})

    return cronograma_mes, sin_agendar_mes

def imprimir_cronograma_anual(cronograma, sin_agendar, anio_inicio, mes_inicio, anio_fin, mes_fin):
    """Imprime el cronograma anual de forma legible en la consola."""
    periodo = f"{mes_inicio}/{anio_inicio} - {mes_fin}/{anio_fin}"
    print("\n" + "="*80)
    print(f"CRONOGRAMA ANUAL DE ENCUENTROS SINCRÓNICOS - PERÍODO {periodo}")
    print("="*80)

    if not cronograma:
        print("No se pudo agendar ningún encuentro.")
        return

    cronograma_ordenado = sorted(cronograma, key=lambda x: (x['fecha'], x['hora_inicio']))

    ultimo_mes_impreso = None
    for encuentro in cronograma_ordenado:
        mes_actual = datetime.datetime.strptime(encuentro['fecha'], '%Y-%m-%d').strftime('%B %Y')
        if mes_actual != ultimo_mes_impreso:
            print(f"\n\n--- {mes_actual.upper()} ---")
            ultimo_mes_impreso = mes_actual

        print(
            f"  [{encuentro['fecha']} {encuentro['hora_inicio']}] "
            f"Grado: {encuentro['grado']:<10} | "
            f"Materia: {encuentro['materia']} (Encuentro #{encuentro['encuentro_nro']}) | "
            f"Instructor: {encuentro['instructor']}"
        )

    if sin_agendar:
        print("\n" + "="*80)
        print("ATENCIÓN: Los siguientes encuentros no pudieron ser agendados:")
        print("="*80)
        for item in sin_agendar:
            print(f"- Mes: {item['mes_fallido']} | Grado: {item['grado']}, Materia: {item['materia']}, Instructor: {item['instructor']}")

def exportar_a_excel_anual(cronograma, sin_agendar, anio_inicio, mes_inicio, anio_fin, mes_fin):
    """Exporta el cronograma anual a un archivo Excel."""
    if not cronograma:
        print("\nNo hay datos para exportar a Excel.")
        return

    datos_para_df = []
    for item in cronograma:
        datos_para_df.append({
            'Fecha': item['fecha'],
            'Hora': f"{item['hora_inicio']} - {item['hora_fin']}",
            'Grado': item['grado'],
            'Materia': item['materia'],
            'Encuentro Nro': item['encuentro_nro'],
            'Instructor': item['instructor']
        })

    df_cronograma = pd.DataFrame(datos_para_df)
    df_cronograma = df_cronograma.sort_values(by=['Fecha', 'Hora']).reset_index(drop=True)

    periodo_str = f"{anio_inicio}-{anio_fin if anio_inicio == anio_fin else str(anio_fin)[-2:]}"
    nombre_archivo = f"Cronograma_Anual_{periodo_str}.xlsx"

    with pd.ExcelWriter(nombre_archivo, engine='openpyxl') as writer:
        df_cronograma.to_excel(writer, sheet_name='Cronograma Anual', index=False)
        if sin_agendar:
            df_sin_agendar = pd.DataFrame(sin_agendar)
            df_sin_agendar.to_excel(writer, sheet_name='No Agendados', index=False)

    print("\n" + "="*80)
    print(f"✅ ¡Cronograma anual exportado exitosamente a '{nombre_archivo}'!")
    print("="*80)

# --- EJECUCIÓN PRINCIPAL ---
if __name__ == "__main__":
    lista_de_materias = [
        {'instructor': 'Yosibel Garcia', 'materia': 'Cívica', 'grado': '7mo grado'},
        {'instructor': 'Ivonne Blanco', 'materia': 'Educación Física', 'grado': '7mo grado'},
        {'instructor': 'Osmary Mujica', 'materia': 'Inglés', 'grado': '7mo grado'},
        {'instructor': 'Freddy Martinez', 'materia': 'Matemática', 'grado': '7no grado'},
        {'instructor': 'Ivonne Blanco', 'materia': 'Ciencias', 'grado': '7mo grado'},
        {'instructor': 'Osmary Mujica', 'materia': 'Castellano', 'grado': '7mo grado'},
        {'instructor': 'Yosibel Garcia', 'materia': 'Lideres Emergentes', 'grado': '7mo grado'},
        {'instructor': 'Jhonfrank Sanchez', 'materia': 'Castellano', 'grado': '8vo grado'},
        {'instructor': 'Jhonfrank Sanchez', 'materia': 'Inglés', 'grado': '8vo grado'},
        {'instructor': 'Freddy Martinez', 'materia': 'Matemática', 'grado': '8vo grado'},
        {'instructor': 'Ivonne Blanco', 'materia': 'Ciencias', 'grado': '8vo grado'},
        {'instructor': 'Alvaro Gonzalez', 'materia': 'Finanzas Personales', 'grado': '8vo grado'},
        {'instructor': 'Ivonne Blanco', 'materia': 'Educación para la Salud', 'grado': '8vo grado'},
        {'instructor': 'Osmary Mujica', 'materia': 'Castellano', 'grado': '9no grado'},
        {'instructor': 'Osmary Mujica', 'materia': 'Inglés', 'grado': '9no grado'},
        {'instructor': 'Freddy Martinez', 'materia': 'Álgebra I', 'grado': '9no grado'},
        {'instructor': 'Ivonne Blanco', 'materia': 'Biología', 'grado': '9no grado'},
        {'instructor': 'Yosibel Garcia', 'materia': 'Historia de USA', 'grado': '9no grado'},
        {'instructor': 'Ivonne Blanco', 'materia': 'Educación Física', 'grado': '9no grado'},
        {'instructor': 'Osmary Mujica', 'materia': 'Inglés', 'grado': '10mo grado'},
        {'instructor': 'Freddy Martinez', 'materia': 'Geometría', 'grado': '10mo grado'},
        {'instructor': 'Yosibel Garcia', 'materia': 'Ética', 'grado': '10mo grado'},
        {'instructor': 'Yosibel Garcia', 'materia': 'Arte', 'grado': '10mo grado'},
        {'instructor': 'Alvaro Gonzalez', 'materia': 'Orientación Vocacional', 'grado': '10mo grado'},
        {'instructor': 'Yosibel Garcia', 'materia': 'Historia Univeral', 'grado': '10mo grado'},
        {'instructor': 'Yosibel Garcia', 'materia': 'Actuación', 'grado': '10mo grado'},
        {'instructor': 'Jhonfrank Sanchez', 'materia': 'Castellano', 'grado': '11vo grado'},
        {'instructor': 'Osmary Mujica', 'materia': 'Inglés', 'grado': '11vo grado'},
        {'instructor': 'Freddy Martinez', 'materia': 'Algebra II', 'grado': '11vo grado'},
        {'instructor': 'Freddy Martinez', 'materia': 'Física', 'grado': '11vo grado'},
        {'instructor': 'Yosibel Garcia', 'materia': 'Arte', 'grado': '11vo grado'},
        {'instructor': 'Alvaro Gonzalez', 'materia': 'Finanzas Personales y Gestión del Dinero', 'grado': '11vo grado'},
        {'instructor': 'Yosibel Garcia', 'materia': 'Historia del Arte', 'grado': '11vo grado'},
        {'instructor': 'Ivonne Blanco', 'materia': 'Ciencias de la Tierra', 'grado': '11vo grado'},
        {'instructor': 'Jhonfrank Sanchez', 'materia': 'Castellano', 'grado': '12vo grado'},
        {'instructor': 'Jhonfrank Sanchez', 'materia': 'Inglés', 'grado': '12vo grado'},
        {'instructor': 'Freddy Martinez', 'materia': 'Ciencias de la Computación', 'grado': '12vo grado'},
        {'instructor': 'Freddy Martinez', 'materia': 'Física', 'grado': '12vo grado'},
        {'instructor': 'Yosibel Garcia', 'materia': 'Gobierno de USA', 'grado': '12vo grado'},
        {'instructor': 'Alvaro Gonzalez', 'materia': 'Liderazgo', 'grado': '12vo grado'},
        {'instructor': 'Alvaro Gonzalez', 'materia': 'Economía', 'grado': '12vo grado'},
        {'instructor': 'Ivonne Blanco', 'materia': 'Química', 'grado': '12vo grado'}
    ]

    # --- Definir el período del año escolar ---
    ANIO_INICIO, MES_INICIO = 2025, 8  # Agosto 2025
    ANIO_FIN, MES_FIN = 2026, 5      # Mayo 2026

    # --- Preparar estructuras para el ciclo anual ---
    cronograma_anual_completo = []
    no_agendados_total = []
    contador_general_encuentros = defaultdict(int) # Usamos defaultdict para simplificar

    print("Iniciando generación de cronograma para el año escolar...")

    # --- Iterar a través de cada mes del año escolar ---
    anio_actual, mes_actual = ANIO_INICIO, MES_INICIO
    while (anio_actual, mes_actual) <= (ANIO_FIN, MES_FIN):
        print(f"\nProcesando mes: {mes_actual}/{anio_actual}...")

        cronograma_del_mes, no_agendados_del_mes = generar_cronograma_mensual(
            lista_de_materias, anio_actual, mes_actual, contador_general_encuentros
        )

        cronograma_anual_completo.extend(cronograma_del_mes)
        no_agendados_total.extend(no_agendados_del_mes)

        # Avanzar al siguiente mes
        mes_actual += 1
        if mes_actual > 12:
            mes_actual = 1
            anio_actual += 1

    # --- Mostrar y exportar resultados finales ---
    imprimir_cronograma_anual(cronograma_anual_completo, no_agendados_total, ANIO_INICIO, MES_INICIO, ANIO_FIN, MES_FIN)
    exportar_a_excel_anual(cronograma_anual_completo, no_agendados_total, ANIO_INICIO, MES_INICIO, ANIO_FIN, MES_FIN)

Iniciando generación de cronograma para el año escolar...

Procesando mes: 8/2025...

Procesando mes: 9/2025...

Procesando mes: 10/2025...

Procesando mes: 11/2025...

Procesando mes: 12/2025...

Procesando mes: 1/2026...

Procesando mes: 2/2026...

Procesando mes: 3/2026...

Procesando mes: 4/2026...

Procesando mes: 5/2026...

CRONOGRAMA ANUAL DE ENCUENTROS SINCRÓNICOS - PERÍODO 8/2025 - 5/2026


--- AUGUST 2025 ---
  [2025-08-01 09:00] Grado: 7mo grado  | Materia: Educación Física (Encuentro #1) | Instructor: Ivonne Blanco
  [2025-08-01 10:00] Grado: 8vo grado  | Materia: Inglés (Encuentro #1) | Instructor: Jhonfrank Sanchez
  [2025-08-01 11:00] Grado: 9no grado  | Materia: Inglés (Encuentro #1) | Instructor: Osmary Mujica
  [2025-08-01 13:00] Grado: 10mo grado | Materia: Geometría (Encuentro #1) | Instructor: Freddy Martinez
  [2025-08-01 14:00] Grado: 11vo grado | Materia: Castellano (Encuentro #1) | Instructor: Jhonfrank Sanchez
  [2025-08-05 09:00] Grado: 7mo grado  | Materia: 

In [ ]:
import calendar
import datetime
import pandas as pd
import os
from collections import defaultdict
import random # Importar la biblioteca random

# Nota: Este script requiere la biblioteca pandas.
# Puedes instalarla con el comando: pip install pandas openpyxl

def generar_cronograma_mensual(materias, anio, mes, contador_encuentros):
    """
    Genera un cronograma de encuentros para un mes específico, actualizando el contador.

    Args:
        materias (list): Lista de materias a agendar para el mes.
        anio (int): El año para el cual se generará el cronograma.
        mes (int): El mes (en número) para el cual se generará el cronograma.
        contador_encuentros (defaultdict): Diccionario para llevar la cuenta de los
                                           encuentros por materia.

    Returns:
        tuple: Una tupla conteniendo dos listas:
               - cronograma_mes (list): Los encuentros agendados para el mes.
               - sin_agendar_mes (list): Las materias que no pudieron ser agendar.
    """
    # --- 1. Definición de Horarios y Constantes ---
    HORAS_DISPONIBLES = [9, 10, 11, 13, 14]
    DIAS_SEMANA = ["Lunes", "Martes", "Miércoles", "Jueves", "Viernes"]

    # --- 2. Inicialización de Estructuras de Datos para el mes ---
    cronograma_mes = []
    sin_agendar_mes = []
    reuniones_instructor_dia = {}
    reuniones_grado_dia = {}
    horarios_ocupados = {}

    # --- 3. Generar la Lista de Días Hábiles del Mes ---
    dias_habiles_del_mes = []
    cal = calendar.Calendar()
    for dia in cal.itermonthdates(anio, mes):
        if dia.month == mes and dia.weekday() < 5:
            dias_habiles_del_mes.append(dia)

    # --- Barajar la lista de días hábiles para distribuir los encuentros ---
    random.shuffle(dias_habiles_del_mes)

    # --- 4. Algoritmo de Asignación Mensual ---
    for materia_info in materias:
        instructor = materia_info['instructor']
        grado = materia_info['grado']
        materia_nombre = materia_info['materia']

        encuentro_agendado = False

        # Iterar sobre los días hábiles (ahora barajados)
        for dia in dias_habiles_del_mes:
            dia_str = dia.strftime('%Y-%m-%d')
            for hora in HORAS_DISPONIBLES:

                if reuniones_instructor_dia.get(instructor, {}).get(dia_str, 0) >= 2:
                    continue
                if reuniones_grado_dia.get(grado, {}).get(dia_str, 0) >= 1:
                    continue
                if hora in horarios_ocupados.get(dia_str, []):
                    continue

                # --- Asignación del Encuentro ---
                # Se encontró un espacio. Se procede a agendar.

                # Incrementar y obtener el número de encuentro para esta materia
                llave_materia = (grado, materia_nombre)
                contador_encuentros[llave_materia] += 1
                numero_encuentro = contador_encuentros[llave_materia]

                cronograma_mes.append({
                    'fecha': dia_str,
                    'dia_semana': DIAS_SEMANA[dia.weekday()],
                    'hora_inicio': f"{hora:02d}:00",
                    'hora_fin': f"{hora+1:02d}:00",
                    'instructor': instructor,
                    'materia': materia_nombre,
                    'grado': grado,
                    'encuentro_nro': numero_encuentro
                })

                # Actualizar estructuras de control
                reuniones_instructor_dia.setdefault(instructor, {}).setdefault(dia_str, 0)
                reuniones_instructor_dia[instructor][dia_str] += 1
                reuniones_grado_dia.setdefault(grado, {}).setdefault(dia_str, 0)
                reuniones_grado_dia[grado][dia_str] += 1
                horarios_ocupados.setdefault(dia_str, []).append(hora)

                encuentro_agendado = True
                break

            if encuentro_agendado:
                break

        if not encuentro_agendado:
            sin_agendar_mes.append({**materia_info, 'mes_fallido': f"{mes}-{anio}"})

    return cronograma_mes, sin_agendar_mes

def imprimir_cronograma_anual(cronograma, sin_agendar, anio_inicio, mes_inicio, anio_fin, mes_fin):
    """Imprime el cronograma anual de forma legible en la consola."""
    periodo = f"{mes_inicio}/{anio_inicio} - {mes_fin}/{anio_fin}"
    print("\n" + "="*80)
    print(f"CRONOGRAMA ANUAL DE ENCUENTROS SINCRÓNICOS - PERÍODO {periodo}")
    print("="*80)

    if not cronograma:
        print("No se pudo agendar ningún encuentro.")
        return

    cronograma_ordenado = sorted(cronograma, key=lambda x: (x['fecha'], x['hora_inicio']))

    ultimo_mes_impreso = None
    for encuentro in cronograma_ordenado:
        mes_actual = datetime.datetime.strptime(encuentro['fecha'], '%Y-%m-%d').strftime('%B %Y')
        if mes_actual != ultimo_mes_impreso:
            print(f"\n\n--- {mes_actual.upper()} ---")
            ultimo_mes_impreso = mes_actual

        print(
            f"  [{encuentro['fecha']} {encuentro['hora_inicio']}] "
            f"Grado: {encuentro['grado']:<10} | "
            f"Materia: {encuentro['materia']} (Encuentro #{encuentro['encuentro_nro']}) | "
            f"Instructor: {encuentro['instructor']}"
        )

    if sin_agendar:
        print("\n" + "="*80)
        print("ATENCIÓN: Los siguientes encuentros no pudieron ser agendados:")
        print("="*80)
        for item in sin_agendar:
            print(f"- Mes: {item['mes_fallido']} | Grado: {item['grado']}, Materia: {item['materia']}, Instructor: {item['instructor']}")

def exportar_a_excel_anual(cronograma, sin_agendar, anio_inicio, mes_inicio, anio_fin, mes_fin):
    """Exporta el cronograma anual a un archivo Excel."""
    if not cronograma:
        print("\nNo hay datos para exportar a Excel.")
        return

    datos_para_df = []
    for item in cronograma:
        datos_para_df.append({
            'Fecha': item['fecha'],
            'Hora': f"{item['hora_inicio']} - {item['hora_fin']}",
            'Grado': item['grado'],
            'Materia': item['materia'],
            'Encuentro Nro': item['encuentro_nro'],
            'Instructor': item['instructor']
        })

    df_cronograma = pd.DataFrame(datos_para_df)
    df_cronograma = df_cronograma.sort_values(by=['Fecha', 'Hora']).reset_index(drop=True)

    periodo_str = f"{anio_inicio}-{anio_fin if anio_inicio == anio_fin else str(anio_fin)[-2:]}"
    nombre_archivo = f"Cronograma_Anual_{periodo_str}.xlsx"

    with pd.ExcelWriter(nombre_archivo, engine='openpyxl') as writer:
        df_cronograma.to_excel(writer, sheet_name='Cronograma Anual', index=False)
        if sin_agendar:
            df_sin_agendar = pd.DataFrame(sin_agendar)
            df_sin_agendar.to_excel(writer, sheet_name='No Agendados', index=False)

    print("\n" + "="*80)
    print(f"✅ ¡Cronograma anual exportado exitosamente a '{nombre_archivo}'!")
    print("="*80)

# --- EJECUCIÓN PRINCIPAL ---
if __name__ == "__main__":
    lista_de_materias = [
        {'instructor': 'Yosibel Garcia', 'materia': 'Cívica', 'grado': '7mo grado'},
        {'instructor': 'Ivonne Blanco', 'materia': 'Educación Física', 'grado': '7mo grado'},
        {'instructor': 'Osmary Mujica', 'materia': 'Inglés', 'grado': '7mo grado'},
        {'instructor': 'Freddy Martinez', 'materia': 'Matemática', 'grado': '7no grado'},
        {'instructor': 'Ivonne Blanco', 'materia': 'Ciencias', 'grado': '7mo grado'},
        {'instructor': 'Osmary Mujica', 'materia': 'Castellano', 'grado': '7mo grado'},
        {'instructor': 'Yosibel Garcia', 'materia': 'Lideres Emergentes', 'grado': '7mo grado'},
        {'instructor': 'Jhonfrank Sanchez', 'materia': 'Castellano', 'grado': '8vo grado'},
        {'instructor': 'Jhonfrank Sanchez', 'materia': 'Inglés', 'grado': '8vo grado'},
        {'instructor': 'Freddy Martinez', 'materia': 'Matemática', 'grado': '8vo grado'},
        {'instructor': 'Ivonne Blanco', 'materia': 'Ciencias', 'grado': '8vo grado'},
        {'instructor': 'Alvaro Gonzalez', 'materia': 'Finanzas Personales', 'grado': '8vo grado'},
        {'instructor': 'Ivonne Blanco', 'materia': 'Educación para la Salud', 'grado': '8vo grado'},
        {'instructor': 'Osmary Mujica', 'materia': 'Castellano', 'grado': '9no grado'},
        {'instructor': 'Osmary Mujica', 'materia': 'Inglés', 'grado': '9no grado'},
        {'instructor': 'Freddy Martinez', 'materia': 'Álgebra I', 'grado': '9no grado'},
        {'instructor': 'Ivonne Blanco', 'materia': 'Biología', 'grado': '9no grado'},
        {'instructor': 'Yosibel Garcia', 'materia': 'Historia de USA', 'grado': '9no grado'},
        {'instructor': 'Ivonne Blanco', 'materia': 'Educación Física', 'grado': '9no grado'},
        {'instructor': 'Osmary Mujica', 'materia': 'Inglés', 'grado': '10mo grado'},
        {'instructor': 'Freddy Martinez', 'materia': 'Geometría', 'grado': '10mo grado'},
        {'instructor': 'Yosibel Garcia', 'materia': 'Ética', 'grado': '10mo grado'},
        {'instructor': 'Yosibel Garcia', 'materia': 'Arte', 'grado': '10mo grado'},
        {'instructor': 'Alvaro Gonzalez', 'materia': 'Orientación Vocacional', 'grado': '10mo grado'},
        {'instructor': 'Yosibel Garcia', 'materia': 'Historia Univeral', 'grado': '10mo grado'},
        {'instructor': 'Yosibel Garcia', 'materia': 'Actuación', 'grado': '10mo grado'},
        {'instructor': 'Jhonfrank Sanchez', 'materia': 'Castellano', 'grado': '11vo grado'},
        {'instructor': 'Osmary Mujica', 'materia': 'Inglés', 'grado': '11vo grado'},
        {'instructor': 'Freddy Martinez', 'materia': 'Algebra II', 'grado': '11vo grado'},
        {'instructor': 'Freddy Martinez', 'materia': 'Física', 'grado': '11vo grado'},
        {'instructor': 'Yosibel Garcia', 'materia': 'Arte', 'grado': '11vo grado'},
        {'instructor': 'Alvaro Gonzalez', 'materia': 'Finanzas Personales y Gestión del Dinero', 'grado': '11vo grado'},
        {'instructor': 'Yosibel Garcia', 'materia': 'Historia del Arte', 'grado': '11vo grado'},
        {'instructor': 'Ivonne Blanco', 'materia': 'Ciencias de la Tierra', 'grado': '11vo grado'},
        {'instructor': 'Jhonfrank Sanchez', 'materia': 'Castellano', 'grado': '12vo grado'},
        {'instructor': 'Jhonfrank Sanchez', 'materia': 'Inglés', 'grado': '12vo grado'},
        {'instructor': 'Freddy Martinez', 'materia': 'Ciencias de la Computación', 'grado': '12vo grado'},
        {'instructor': 'Freddy Martinez', 'materia': 'Física', 'grado': '12vo grado'},
        {'instructor': 'Yosibel Garcia', 'materia': 'Gobierno de USA', 'grado': '12vo grado'},
        {'instructor': 'Alvaro Gonzalez', 'materia': 'Liderazgo', 'grado': '12vo grado'},
        {'instructor': 'Alvaro Gonzalez', 'materia': 'Economía', 'grado': '12vo grado'},
        {'instructor': 'Ivonne Blanco', 'materia': 'Química', 'grado': '12vo grado'}
    ]

    # --- Definir el período del año escolar ---
    ANIO_INICIO, MES_INICIO = 2025, 8  # Agosto 2025
    ANIO_FIN, MES_FIN = 2026, 5      # Mayo 2026

    # --- Preparar estructuras para el ciclo anual ---
    cronograma_anual_completo = []
    no_agendados_total = []
    contador_general_encuentros = defaultdict(int) # Usamos defaultdict para simplificar

    print("Iniciando generación de cronograma para el año escolar...")

    # --- Iterar a través de cada mes del año escolar ---
    anio_actual, mes_actual = ANIO_INICIO, MES_INICIO
    while (anio_actual, mes_actual) <= (ANIO_FIN, MES_FIN):
        print(f"\nProcesando mes: {mes_actual}/{anio_actual}...")

        cronograma_del_mes, no_agendados_del_mes = generar_cronograma_mensual(
            lista_de_materias, anio_actual, mes_actual, contador_general_encuentros
        )

        cronograma_anual_completo.extend(cronograma_del_mes)
        no_agendados_total.extend(no_agendados_del_mes)

        # Avanzar al siguiente mes
        mes_actual += 1
        if mes_actual > 12:
            mes_actual = 1
            anio_actual += 1

    # --- Mostrar y exportar resultados finales ---
    imprimir_cronograma_anual(cronograma_anual_completo, no_agendados_total, ANIO_INICIO, MES_INICIO, ANIO_FIN, MES_FIN)
    exportar_a_excel_anual(cronograma_anual_completo, no_agendados_total, ANIO_INICIO, MES_INICIO, ANIO_FIN, MES_FIN)

Iniciando generación de cronograma para el año escolar...

Procesando mes: 8/2025...

Procesando mes: 9/2025...

Procesando mes: 10/2025...

Procesando mes: 11/2025...

Procesando mes: 12/2025...

Procesando mes: 1/2026...

Procesando mes: 2/2026...

Procesando mes: 3/2026...

Procesando mes: 4/2026...

Procesando mes: 5/2026...

CRONOGRAMA ANUAL DE ENCUENTROS SINCRÓNICOS - PERÍODO 8/2025 - 5/2026


--- AUGUST 2025 ---
  [2025-08-01 09:00] Grado: 12vo grado | Materia: Economía (Encuentro #1) | Instructor: Alvaro Gonzalez
  [2025-08-05 09:00] Grado: 12vo grado | Materia: Liderazgo (Encuentro #1) | Instructor: Alvaro Gonzalez
  [2025-08-06 09:00] Grado: 7mo grado  | Materia: Educación Física (Encuentro #1) | Instructor: Ivonne Blanco
  [2025-08-06 10:00] Grado: 8vo grado  | Materia: Inglés (Encuentro #1) | Instructor: Jhonfrank Sanchez
  [2025-08-06 11:00] Grado: 9no grado  | Materia: Inglés (Encuentro #1) | Instructor: Osmary Mujica
  [2025-08-06 13:00] Grado: 10mo grado | Materia: Geom

In [ ]:
import calendar
import datetime
import pandas as pd
import os
from collections import defaultdict
import random # Importar la biblioteca random

# Nota: Este script requiere la biblioteca pandas.
# Puedes instalarla con el comando: pip install pandas openpyxl

def generar_cronograma_mensual(materias, anio, mes, contador_encuentros):
    """
    Genera un cronograma de encuentros para un mes específico, actualizando el contador.

    Args:
        materias (list): Lista de materias a agendar para el mes.
        anio (int): El año para el cual se generará el cronograma.
        mes (int): El mes (en número) para el cual se generará el cronograma.
        contador_encuentros (defaultdict): Diccionario para llevar la cuenta de los
                                           encuentros por materia.

    Returns:
        tuple: Una tupla conteniendo dos listas:
               - cronograma_mes (list): Los encuentros agendados para el mes.
               - sin_agendar_mes (list): Las materias que no pudieron ser agendar.
    """
    # --- 1. Definición de Horarios y Constantes ---
    HORAS_DISPONIBLES = [9, 10, 11, 13, 14, 15]
    DIAS_SEMANA = ["Lunes", "Martes", "Miércoles", "Jueves", "Viernes"]

    # --- 2. Inicialización de Estructuras de Datos para el mes ---
    cronograma_mes = []
    sin_agendar_mes = []
    reuniones_instructor_dia = {}
    reuniones_grado_dia = {}
    horarios_ocupados = {}

    # --- 3. Generar la Lista de Días Hábiles del Mes ---
    dias_habiles_del_mes = []
    cal = calendar.Calendar()
    for dia in cal.itermonthdates(anio, mes):
        if dia.month == mes and dia.weekday() < 5:
            dias_habiles_del_mes.append(dia)

    # --- Barajar la lista de días hábiles para distribuir los encuentros ---
    random.shuffle(dias_habiles_del_mes)

    # --- 4. Algoritmo de Asignación Mensual ---
    for materia_info in materias:
        instructor = materia_info['instructor']
        grado = materia_info['grado']
        materia_nombre = materia_info['materia']

        encuentro_agendado = False

        # Iterar sobre los días hábiles (ahora barajados)
        for dia in dias_habiles_del_mes:
            dia_str = dia.strftime('%Y-%m-%d')
            for hora in HORAS_DISPONIBLES:

                if reuniones_instructor_dia.get(instructor, {}).get(dia_str, 0) >= 2:
                    continue
                if reuniones_grado_dia.get(grado, {}).get(dia_str, 0) >= 1:
                    continue
                if hora in horarios_ocupados.get(dia_str, []):
                    continue

                # --- Asignación del Encuentro ---
                # Se encontró un espacio. Se procede a agendar.

                # Incrementar y obtener el número de encuentro para esta materia
                llave_materia = (grado, materia_nombre)
                contador_encuentros[llave_materia] += 1
                numero_encuentro = contador_encuentros[llave_materia]

                cronograma_mes.append({
                    'fecha': dia_str,
                    'dia_semana': DIAS_SEMANA[dia.weekday()],
                    'hora_inicio': f"{hora:02d}:00",
                    'hora_fin': f"{hora+1:02d}:00",
                    'instructor': instructor,
                    'materia': materia_nombre,
                    'grado': grado,
                    'encuentro_nro': numero_encuentro
                })

                # Actualizar estructuras de control
                reuniones_instructor_dia.setdefault(instructor, {}).setdefault(dia_str, 0)
                reuniones_instructor_dia[instructor][dia_str] += 1
                reuniones_grado_dia.setdefault(grado, {}).setdefault(dia_str, 0)
                reuniones_grado_dia[grado][dia_str] += 1
                horarios_ocupados.setdefault(dia_str, []).append(hora)

                encuentro_agendado = True
                break

            if encuentro_agendado:
                break

        if not encuentro_agendado:
            sin_agendar_mes.append({**materia_info, 'mes_fallido': f"{mes}-{anio}"})

    return cronograma_mes, sin_agendar_mes

def imprimir_cronograma_anual(cronograma, sin_agendar, anio_inicio, mes_inicio, anio_fin, mes_fin):
    """Imprime el cronograma anual de forma legible en la consola."""
    periodo = f"{mes_inicio}/{anio_inicio} - {mes_fin}/{anio_fin}"
    print("\n" + "="*80)
    print(f"CRONOGRAMA ANUAL DE ENCUENTROS SINCRÓNICOS - PERÍODO {periodo}")
    print("="*80)

    if not cronograma:
        print("No se pudo agendar ningún encuentro.")
        return

    cronograma_ordenado = sorted(cronograma, key=lambda x: (x['fecha'], x['hora_inicio']))

    ultimo_mes_impreso = None
    for encuentro in cronograma_ordenado:
        mes_actual = datetime.datetime.strptime(encuentro['fecha'], '%Y-%m-%d').strftime('%B %Y')
        if mes_actual != ultimo_mes_impreso:
            print(f"\n\n--- {mes_actual.upper()} ---")
            ultimo_mes_impreso = mes_actual

        print(
            f"  [{encuentro['fecha']} {encuentro['hora_inicio']}] "
            f"Grado: {encuentro['grado']:<10} | "
            f"Materia: {encuentro['materia']} (Encuentro #{encuentro['encuentro_nro']}) | "
            f"Instructor: {encuentro['instructor']}"
        )

    if sin_agendar:
        print("\n" + "="*80)
        print("ATENCIÓN: Los siguientes encuentros no pudieron ser agendados:")
        print("="*80)
        for item in sin_agendar:
            print(f"- Mes: {item['mes_fallido']} | Grado: {item['grado']}, Materia: {item['materia']}, Instructor: {item['instructor']}")

def exportar_a_excel_anual(cronograma, sin_agendar, anio_inicio, mes_inicio, anio_fin, mes_fin):
    """Exporta el cronograma anual a un archivo Excel."""
    if not cronograma:
        print("\nNo hay datos para exportar a Excel.")
        return

    datos_para_df = []
    for item in cronograma:
        datos_para_df.append({
            'Fecha': item['fecha'],
            'Hora': f"{item['hora_inicio']} - {item['hora_fin']}",
            'Grado': item['grado'],
            'Materia': item['materia'],
            'Encuentro Nro': item['encuentro_nro'],
            'Instructor': item['instructor']
        })

    df_cronograma = pd.DataFrame(datos_para_df)
    df_cronograma = df_cronograma.sort_values(by=['Fecha', 'Hora']).reset_index(drop=True)

    periodo_str = f"{anio_inicio}-{anio_fin if anio_inicio == anio_fin else str(anio_fin)[-2:]}"
    nombre_archivo = f"Cronograma_Anual_{periodo_str}.xlsx"

    with pd.ExcelWriter(nombre_archivo, engine='openpyxl') as writer:
        df_cronograma.to_excel(writer, sheet_name='Cronograma Anual', index=False)
        if sin_agendar:
            df_sin_agendar = pd.DataFrame(sin_agendar)
            df_sin_agendar.to_excel(writer, sheet_name='No Agendados', index=False)

    print("\n" + "="*80)
    print(f"✅ ¡Cronograma anual exportado exitosamente a '{nombre_archivo}'!")
    print("="*80)

# --- EJECUCIÓN PRINCIPAL ---
if __name__ == "__main__":
    lista_de_materias = [
        {'instructor': 'Yosibel Garcia', 'materia': 'Cívica', 'grado': '7mo grado'},
        {'instructor': 'Ivonne Blanco', 'materia': 'Educación Física', 'grado': '7mo grado'},
        {'instructor': 'Osmary Mujica', 'materia': 'Inglés', 'grado': '7mo grado'},
        {'instructor': 'Freddy Martinez', 'materia': 'Matemática', 'grado': '7no grado'},
        {'instructor': 'Ivonne Blanco', 'materia': 'Ciencias', 'grado': '7mo grado'},
        {'instructor': 'Osmary Mujica', 'materia': 'Castellano', 'grado': '7mo grado'},
        {'instructor': 'Yosibel Garcia', 'materia': 'Lideres Emergentes', 'grado': '7mo grado'},
        {'instructor': 'Jhonfrank Sanchez', 'materia': 'Castellano', 'grado': '8vo grado'},
        {'instructor': 'Jhonfrank Sanchez', 'materia': 'Inglés', 'grado': '8vo grado'},
        {'instructor': 'Freddy Martinez', 'materia': 'Matemática', 'grado': '8vo grado'},
        {'instructor': 'Ivonne Blanco', 'materia': 'Ciencias', 'grado': '8vo grado'},
        {'instructor': 'Alvaro Gonzalez', 'materia': 'Finanzas Personales', 'grado': '8vo grado'},
        {'instructor': 'Ivonne Blanco', 'materia': 'Educación para la Salud', 'grado': '8vo grado'},
        {'instructor': 'Osmary Mujica', 'materia': 'Castellano', 'grado': '9no grado'},
        {'instructor': 'Osmary Mujica', 'materia': 'Inglés', 'grado': '9no grado'},
        {'instructor': 'Freddy Martinez', 'materia': 'Álgebra I', 'grado': '9no grado'},
        {'instructor': 'Ivonne Blanco', 'materia': 'Biología', 'grado': '9no grado'},
        {'instructor': 'Yosibel Garcia', 'materia': 'Historia de USA', 'grado': '9no grado'},
        {'instructor': 'Ivonne Blanco', 'materia': 'Educación Física', 'grado': '9no grado'},
        {'instructor': 'Osmary Mujica', 'materia': 'Inglés', 'grado': '10mo grado'},
        {'instructor': 'Freddy Martinez', 'materia': 'Geometría', 'grado': '10mo grado'},
        {'instructor': 'Yosibel Garcia', 'materia': 'Ética', 'grado': '10mo grado'},
        {'instructor': 'Yosibel Garcia', 'materia': 'Arte', 'grado': '10mo grado'},
        {'instructor': 'Alvaro Gonzalez', 'materia': 'Orientación Vocacional', 'grado': '10mo grado'},
        {'instructor': 'Yosibel Garcia', 'materia': 'Historia Univeral', 'grado': '10mo grado'},
        {'instructor': 'Yosibel Garcia', 'materia': 'Actuación', 'grado': '10mo grado'},
        {'instructor': 'Jhonfrank Sanchez', 'materia': 'Castellano', 'grado': '11vo grado'},
        {'instructor': 'Osmary Mujica', 'materia': 'Inglés', 'grado': '11vo grado'},
        {'instructor': 'Freddy Martinez', 'materia': 'Algebra II', 'grado': '11vo grado'},
        {'instructor': 'Freddy Martinez', 'materia': 'Física', 'grado': '11vo grado'},
        {'instructor': 'Yosibel Garcia', 'materia': 'Arte', 'grado': '11vo grado'},
        {'instructor': 'Alvaro Gonzalez', 'materia': 'Finanzas Personales y Gestión del Dinero', 'grado': '11vo grado'},
        {'instructor': 'Yosibel Garcia', 'materia': 'Historia del Arte', 'grado': '11vo grado'},
        {'instructor': 'Ivonne Blanco', 'materia': 'Ciencias de la Tierra', 'grado': '11vo grado'},
        {'instructor': 'Jhonfrank Sanchez', 'materia': 'Castellano', 'grado': '12vo grado'},
        {'instructor': 'Jhonfrank Sanchez', 'materia': 'Inglés', 'grado': '12vo grado'},
        {'instructor': 'Freddy Martinez', 'materia': 'Ciencias de la Computación', 'grado': '12vo grado'},
        {'instructor': 'Freddy Martinez', 'materia': 'Física', 'grado': '12vo grado'},
        {'instructor': 'Yosibel Garcia', 'materia': 'Gobierno de USA', 'grado': '12vo grado'},
        {'instructor': 'Alvaro Gonzalez', 'materia': 'Liderazgo', 'grado': '12vo grado'},
        {'instructor': 'Alvaro Gonzalez', 'materia': 'Economía', 'grado': '12vo grado'},
        {'instructor': 'Ivonne Blanco', 'materia': 'Química', 'grado': '12vo grado'}
    ]

    # --- Definir el período del año escolar ---
    ANIO_INICIO, MES_INICIO = 2025, 8  # Agosto 2025
    ANIO_FIN, MES_FIN = 2026, 5      # Mayo 2026

    # --- Preparar estructuras para el ciclo anual ---
    cronograma_anual_completo = []
    no_agendados_total = []
    contador_general_encuentros = defaultdict(int) # Usamos defaultdict para simplificar

    print("Iniciando generación de cronograma para el año escolar...")

    # --- Iterar a través de cada mes del año escolar ---
    anio_actual, mes_actual = ANIO_INICIO, MES_INICIO
    while (anio_actual, mes_actual) <= (ANIO_FIN, MES_FIN):
        print(f"\nProcesando mes: {mes_actual}/{anio_actual}...")

        cronograma_del_mes, no_agendados_del_mes = generar_cronograma_mensual(
            lista_de_materias, anio_actual, mes_actual, contador_general_encuentros
        )

        cronograma_anual_completo.extend(cronograma_del_mes)
        no_agendados_total.extend(no_agendados_del_mes)

        # Avanzar al siguiente mes
        mes_actual += 1
        if mes_actual > 12:
            mes_actual = 1
            anio_actual += 1

    # --- Mostrar y exportar resultados finales ---
    imprimir_cronograma_anual(cronograma_anual_completo, no_agendados_total, ANIO_INICIO, MES_INICIO, ANIO_FIN, MES_FIN)
    exportar_a_excel_anual(cronograma_anual_completo, no_agendados_total, ANIO_INICIO, MES_INICIO, ANIO_FIN, MES_FIN)

Iniciando generación de cronograma para el año escolar...

Procesando mes: 8/2025...

Procesando mes: 9/2025...

Procesando mes: 10/2025...

Procesando mes: 11/2025...

Procesando mes: 12/2025...

Procesando mes: 1/2026...

Procesando mes: 2/2026...

Procesando mes: 3/2026...

Procesando mes: 4/2026...

Procesando mes: 5/2026...

CRONOGRAMA ANUAL DE ENCUENTROS SINCRÓNICOS - PERÍODO 8/2025 - 5/2026


--- AUGUST 2025 ---
  [2025-08-01 09:00] Grado: 11vo grado | Materia: Ciencias de la Tierra (Encuentro #1) | Instructor: Ivonne Blanco
  [2025-08-01 10:00] Grado: 12vo grado | Materia: Ciencias de la Computación (Encuentro #1) | Instructor: Freddy Martinez
  [2025-08-05 09:00] Grado: 12vo grado | Materia: Economía (Encuentro #1) | Instructor: Alvaro Gonzalez
  [2025-08-06 09:00] Grado: 12vo grado | Materia: Física (Encuentro #1) | Instructor: Freddy Martinez
  [2025-08-11 09:00] Grado: 12vo grado | Materia: Química (Encuentro #1) | Instructor: Ivonne Blanco
  [2025-08-12 09:00] Grado: 7mo g